In [21]:
import cv2 as cv
import pandas as pd
import glob
import torch
import random
import time
import os
import csv
import numpy as np
import tensorflow as tf
import pdb
import matplotlib.pyplot as plt
import codecs

from facenet_pytorch import MTCNN
from tensorflow import keras
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from facenet_pytorch import MTCNN, InceptionResnetV1
from tensorflow.keras.models import load_model

In [6]:
train_small_csv_paths = 'C:\\Users\\96291\\Desktop\\Mini_challenge\\un_ziped\\purdue-face-recognition-challenge-2024\\train.csv'
# contains training images with file names of the form n.jpg
category_path = 'C:\\Users\\96291\\Desktop\\Mini_challenge\\un_ziped\\purdue-face-recognition-challenge-2024\\category.csv'


train_data = pd.read_csv(train_small_csv_paths, encoding = 'latin1') 
numerical_labels = pd.read_csv(category_path, encoding = 'ISO-8859-1')
print(numerical_labels)

# Create a dictionary for category to numerical label mapping
#categpry should be at lefe, because use category to find the numerical_labels. 
category_to_label = dict(zip( numerical_labels['Category'], numerical_labels['Unnamed: 0']))

# Load image data and replace string labels with numerical labels
train_data['NumericalLabel'] = train_data['Category'].map(category_to_label)
print(train_data)


    Unnamed: 0            Category
0            0          Art Carney
1            1    Akemi Darenogare
2            2  Billy Bob Thornton
3            3          Bill Daley
4            4        Amanda Bynes
..         ...                 ...
95          95          Bill Hader
96          96         Ben Johnson
97          97         Ava Gardner
98          98               Adele
99          99        Bette Midler

[100 rows x 2 columns]
       Unnamed: 0  File Name         Category  NumericalLabel
0               0      0.jpg    Audrey Tautou              87
1               1      1.jpg     Adam Sandler              90
2               2      2.jpg      Anna Paquin              39
3               3      3.jpg      Ava Gardner              97
4               4      4.jpg        Amy Adams              30
...           ...        ...              ...             ...
69535       69535  69535.jpg         Anna Sui              17
69536       69536  69536.jpg  AnnaSophia Robb              8

In [12]:
print(train_data)

       Unnamed: 0  File Name         Category  NumericalLabel
0               0      0.jpg    Audrey Tautou              87
1               1      1.jpg     Adam Sandler              90
2               2      2.jpg      Anna Paquin              39
3               3      3.jpg      Ava Gardner              97
4               4      4.jpg        Amy Adams              30
...           ...        ...              ...             ...
69535       69535  69535.jpg         Anna Sui              17
69536       69536  69536.jpg  AnnaSophia Robb              85
69537       69537  69537.jpg  Beyonce Knowles              92
69538       69538  69538.jpg    Alexis Thorpe              19
69539       69539  69539.jpg      Amanda Peet              24

[69540 rows x 4 columns]


In [7]:
# create labels for each file in cvs file
train_y = train_data['NumericalLabel']
print(train_y)

0        87
1        90
2        39
3        97
4        30
         ..
69535    17
69536    85
69537    92
69538    19
69539    24
Name: NumericalLabel, Length: 69540, dtype: int64


### Crop face

In [8]:
import torch
from facenet_pytorch import MTCNN
from PIL import Image
import os
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=False, device=device)

small_num_samples = 500
num_samples = train_y.shape[0]
print(num_samples)
source_folder = 'C:\\Users\\96291\\Desktop\\Mini_challenge\\un_ziped\\train'
output_folder = 'C:\\Users\\96291\\Desktop\\Mini_challenge\\un_ziped\\test_croped_face'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i in range(num_samples):
    image_path = f'{source_folder}/{i}.jpg'
    img = Image.open(image_path)

    # Skip the image if it's blank or nearly blank
    if np.std(np.array(img)) < 1e-6:  # Threshold can be adjusted
        print(f'Skipping blank image {i}')
        continue

    if img.mode != 'RGB':
        img = img.convert('RGB')

    box, _ = mtcnn.detect(img)

    if box is not None:
        box = box[0]  # Since keep_all=False, only one box is returned
        crop = img.crop(box)
        crop.save(os.path.join(output_folder, f'{i}.jpg'))

    print(f'Processed image {i}')

69540
Processed image 0
Processed image 1
Processed image 2
Processed image 3
Processed image 4
Processed image 5
Processed image 6
Processed image 7
Processed image 8
Processed image 9
Processed image 10
Processed image 11
Processed image 12
Processed image 13
Processed image 14


KeyboardInterrupt: 

In [14]:
#Prepare the data
train_data, test_data = train_test_split(train_data, test_size=0.2, random_state=42)


# Prepare the data generators. 
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='C:\\Users\\96291\\Desktop\\Mini_challenge\\un_ziped\\test_croped_face',  # Update this path to where your images are stored
    x_col='File Name',  # or whatever the exact column name is for filenames in train_df
    y_col='NumericalLabel',  # or the column name for numerical labels in category_df
    target_size=(128, 128),
    batch_size=32,
    class_mode='raw'  # use 'raw' since you have numerical labels
)


Found 54412 validated image filenames.


C:\Users\96291\AppData\Roaming\Python\Python312\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 1220 invalid image filename(s) in x_col="File Name". These filename(s) will be ignored.
  warnings.warn(


In [22]:
face_recg = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(100, activation='softmax')  # Assuming 100 classes from 0-99
])

face_recg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
face_recg.fit(train_generator, epochs=10)

Epoch 1/10
1701/1701 ━━━━━━━━━━━━━━━━━━━━ 249s 145ms/step - accuracy: 0.0688 - loss: 4.2875
Epoch 2/10
1701/1701 ━━━━━━━━━━━━━━━━━━━━ 240s 141ms/step - accuracy: 0.3210 - loss: 3.0217
Epoch 3/10
1701/1701 ━━━━━━━━━━━━━━━━━━━━ 239s 140ms/step - accuracy: 0.4662 - loss: 2.3342
Epoch 4/10
1701/1701 ━━━━━━━━━━━━━━━━━━━━ 241s 142ms/step - accuracy: 0.5676 - loss: 1.8506
Epoch 5/10
1701/1701 ━━━━━━━━━━━━━━━━━━━━ 241s 142ms/step - accuracy: 0.6418 - loss: 1.4799
Epoch 6/10
1701/1701 ━━━━━━━━━━━━━━━━━━━━ 240s 141ms/step - accuracy: 0.6992 - loss: 1.2124
Epoch 7/10
1701/1701 ━━━━━━━━━━━━━━━━━━━━ 240s 141ms/step - accuracy: 0.7376 - loss: 1.0148
Epoch 8/10
1701/1701 ━━━━━━━━━━━━━━━━━━━━ 240s 141ms/step - accuracy: 0.7741 - loss: 0.8410
Epoch 9/10
1701/1701 ━━━━━━━━━━━━━━━━━━━━ 240s 141ms/step - accuracy: 0.7980 - loss: 0.7441
Epoch 10/10
1701/1701 ━━━━━━━━━━━━━━━━━━━━ 241s 141ms/step - accuracy: 0.8256 - loss: 0.6346


In [26]:
# Load the trained model
model = face_recg

# Initialize MTCNN for face detection
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=False, device=device)

# Directory containing test images
test_images_folder = 'C:\\Users\\96291\\Desktop\\Mini_challenge\\un_ziped\\test'

# Load category names from your category mapping file
category_df = pd.read_csv('C:\\Users\\96291\\Desktop\\Mini_challenge\\un_ziped\\purdue-face-recognition-challenge-2024\\category.csv')
category_mapping = dict(zip(category_df['Unnamed: 0'], category_df['Category']))

# Process images and classify
results = []
# test_images = sorted(os.listdir(test_images_folder))  # Sort to ensure order
test_images = sorted([file for file in os.listdir(test_images_folder) if not file.startswith('._')])

for img_name in test_images:
    if img_name.lower().endswith('.jpg'):
        img_path = os.path.join(test_images_folder, img_name)
        img = Image.open(img_path).convert('RGB')

        # Detect the face
        boxes, _ = mtcnn.detect(img)
        if boxes is not None and len(boxes) > 0:
            # Crop the detected face
            face = img.crop(boxes[0])
            face = face.resize((128, 128))
            face_array = np.array(face) / 255.0
            face_array = np.expand_dims(face_array, axis=0)

            # Classify the face
            prediction = model.predict(face_array)
            predicted_index = np.argmax(prediction, axis=1)[0]
            predicted_category = category_mapping.get(predicted_index, 'Unknown')
        else:
            predicted_category = 'Unknown'

        file_id = int(os.path.splitext(img_name)[0])  # Convert filename to integer for sorting
        results.append({'Id': file_id, 'Category': predicted_category})



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

C:\Users\96291\AppData\Roaming\Python\Python312\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━

In [27]:
# Sort results by 'Id' to ensure correct order
results_df = pd.DataFrame(results).sort_values(by='Id')
results_df['Id'] = results_df['Id'].astype(str)  # Convert back to string if necessary

# Save the DataFrame to a CSV file
results_df.to_csv('C:\\Users\\96291\\Desktop\\Mini_challenge\\un_ziped\\predict.csv', index=False)